<a href="https://colab.research.google.com/github/vitor-yuichi/Tweet_Pluviometer/blob/main/Files/Hypothesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
from scipy import stats 
from google.colab import drive
drive.mount('/content/gdrive') 
import numpy as np

Mounted at /content/gdrive


# Shapiro WilK test 

In [ ]:
#archive_flood=pd.read_csv(r'/content/drive/Shareddrives/Colabs/Colab Notebooks/Tweets/arquivo1.csv') #Temp. path
archive_flood=pd.read_csv(r'/content/gdrive/My Drive/Colab Notebooks/Tweets/arquivo1.csv')
archive_flood

,index,Tweets frequence,Floods frequency,Duration,METEO,HIDRO
0,2019-03-02,4,3.0,1.59,4,0
1,2019-03-03,3,0.0,0.00,4,0
2,2019-03-04,1,0.0,0.00,1,0
3,2019-03-05,3,5.0,13.56,3,0
4,2019-03-06,1,0.0,0.00,2,0
5,2019-03-07,9,6.0,9.39,10,0
6,2019-03-09,11,2.0,0.60,12,0
7,2019-03-10,2,0.0,0.00,2,0
8,2019-03-11,7,0.0,0.00,8,0
9,2019-03-12,6,0.0,0.00,7,2


## Shapiro-Wilk's concept 
This statistical method is a test of normality. It is widely used in many other tests.


1.   The Shapiro-Wilk test tests the null hypothesis that a sample y1,y2,⋯,yn, taken from a population, has a normal distribution.
2.   Reject the null hypothesis if p<alpha \\

Assuming alpha = 0.05  
If we reject the null hypothesis, we conclude that out variable is not normally distributed. 















In [ ]:
flood_days=archive_flood[archive_flood['Floods frequency']!=0]
no_flood=archive_flood[archive_flood['Floods frequency']==0]
no_flood, flood_days

(        index  Tweets frequence  Floods frequency  Duration  METEO  HIDRO
 1  2019-03-03                 3               0.0       0.0      4      0
 2  2019-03-04                 1               0.0       0.0      1      0
 4  2019-03-06                 1               0.0       0.0      2      0
 7  2019-03-10                 2               0.0       0.0      2      0
 8  2019-03-11                 7               0.0       0.0      8      0
 9  2019-03-12                 6               0.0       0.0      7      2,
         index  Tweets frequence  Floods frequency  Duration  METEO  HIDRO
 0  2019-03-02                 4               3.0      1.59      4      0
 3  2019-03-05                 3               5.0     13.56      3      0
 5  2019-03-07                 9               6.0      9.39     10      0
 6  2019-03-09                11               2.0      0.60     12      0)

## Problema:
* Recomendo a leitura deste artigo: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3693611/
* Atenção a citação: "For small sample sizes, normality tests have little power to reject the null hypothesis and therefore small samples most often pass normality tests"
* Devido a quantidade reduzida de observações, um teste paramétrico não tem poder explicativo

In [ ]:
#Check the sample sizes 
print('flood/non-flood:',flood_days.shape[0],'/',no_flood.shape[0])

flood/non-flood: 4 / 6


In [ ]:
#flood days 
shapiro_stat, shapiro_p_value = stats.shapiro(flood_days['METEO'].array)
print("Statistics of Shapiro-Wilk in flood days = "+ str(shapiro_stat))
print("p-value (flood day) = "+ str(shapiro_p_value))

#no flood days 
shapiro_stat1, shapiro_p_value1=stats.shapiro(no_flood['METEO'].array)
print("Statistics of Shapiro-Wilk in no floods day = "+ str(shapiro_stat1))
print("p-value (no flood) = "+ str(shapiro_p_value1))


#Segundo as definições anteriores, é bem provavel que o SW implique normalidade... apesar de nãos ser

Statistics of Shapiro-Wilk in flood days = 0.8783182501792908
p-value (flood day) = 0.3315027952194214
Statistics of Shapiro-Wilk in no floods day = 0.8804032206535339
p-value (no flood) = 0.2709139883518219


In [ ]:
# testing the hypothesis assuming alpha = 0.05
if shapiro_p_value>0.05: 
  print('The p value for flood days is ' + str(shapiro_p_value))
  print( 'The p value is greater than alpha, so the hypothesis cannot be rejected')
else:
  print('Reject the null hypothesis')
if shapiro_p_value1>0.05:
  print('The p value for no flood days is ' + str(shapiro_p_value1))
  print( 'The p value is greater than alpha, so the hypothesis cannot be rejected')
else:
  print('Reject the null hypothesis')

The p value for flood days is 0.3315027952194214
The p value is greater than alpha, so the hypothesis cannot be rejected
The p value for no flood days is 0.2709139883518219
The p value is greater than alpha, so the hypothesis cannot be rejected


# MannWhitney test 


## Mann-Whitney concept 
The Mann–Whitney test is a nonparametric test that aims to test the equality of two populations. It is used when we have two samples coming from two populations.
The null hypothesis that, for randomly selected values X and Y from two populations, the probability of X being greater than Y is equal to the probability of Y being greater than X.

if (p-value<alpha), reject the null hypothesis 

Reject the null hypothesis 

In [ ]:

x,y=stats.mannwhitneyu(flood_days['METEO'].array, no_flood['METEO'].array,alternative='two-sided')
print("The statistics: ", x)
print("The p-value", y)
if y<=0.05:
  print("Reject the null hypothesis, that the probability of X being greater than Y is equal to the probability of Y being greater than X")
else:
  print("The null hypothesis cannot be rejected")

The statistics:  18.5
The p-value 0.19808444160085592
The null hypothesis cannot be rejected


In [ ]:
#Ajustes/comentários

#... annwhitneyu always reports the statistic associated with the first sample, which, in this case, is males.
#Em resumo, a comparação aqui é de flood_days em relação a no_flood (pois vem primeiro na sua lista de parâmetros)
#Acredito que esta seja a ordem ideal, pois temos um conhecimento a priori que nos dias de alagamento a frequência de tweets é maior
#Com isso, admitindo que um é maior que o outro, um teste unilateral ('greater') faz mais sentido (deixa o teste ainda mais robusto...)

x,y=stats.mannwhitneyu(flood_days['METEO'].array, no_flood['METEO'].array, alternative='greater')
print("The statistics: ", x)
print("The p-value", y)
if y<=0.05:
  print("Reject the null hypothesis, that the probability of X being greater than Y is equal to the probability of Y being greater than X")
else:
  print("The null hypothesis cannot be rejected")

The statistics:  18.5
The p-value 0.09904222080042796
The null hypothesis cannot be rejected


In [ ]:
#Teste de Kruskal-Wallis
#Este teste é indicado para duas ou mais amostras -- incluí aqui apenas por teste...

In [ ]:
from scipy.stats import kruskal
stat, p = kruskal(flood_days['METEO'].array, no_flood['METEO'].array)
print("The statistics: ", stat)
print("The p-value", p)
if y<=0.05:
  print("Reject the null hypothesis, that the probability of X being greater than Y is equal to the probability of Y being greater than X")
else:
  print("The null hypothesis cannot be rejected")

The statistics:  1.9440184049079767
The p-value 0.1632328854401128
The null hypothesis cannot be rejected


#Second Temporal Window

In [ ]:
second_tw=pd.read_csv(r'/content/gdrive/My Drive/Colab Notebooks/Tweets//Second_Twindow.csv')
second_tw

,index,Tweets Frequence,Floods Frequence,Duration,METEO,HIDRO
0,2019-01-03,2,0.0,0.00,1,1
1,2019-02-03,6,3.0,1.59,6,0
2,2019-03-03,5,0.0,0.00,6,0
3,2019-04-03,1,0.0,0.00,1,0
4,2019-05-03,2,5.0,13.56,2,0
5,2019-06-03,1,0.0,0.00,2,0
6,2019-07-03,8,6.0,9.39,9,0
7,2019-09-03,11,2.0,0.60,12,0
8,2019-10-03,2,0.0,0.00,2,0
9,2019-11-03,7,0.0,0.00,8,0


In [ ]:
flood_days=second_tw[second_tw['Floods Frequence']!=0]
no_flood=second_tw[second_tw['Floods Frequence']==0]

In [ ]:
x,y=stats.mannwhitneyu(flood_days['METEO'].array, no_flood['METEO'].array, alternative='greater')
print("The statistics: ", x)
print("The p-value", y)
if y<=0.05:
  print("Reject the null hypothesis, that the probability of X being greater than Y is equal to the probability of Y being greater than X")
else:
  print("The null hypothesis cannot be rejected")

The statistics:  49.0
The p-value 0.05725926424753307
The null hypothesis cannot be rejected


# Third Window 

In [ ]:
third_tw=pd.read_csv(r'/content/gdrive/My Drive/Colab Notebooks/Tweets/Third_Twindow.csv')
third_tw

,index,Tweets Frequence,Floods Frequence,Duration,METEO
0,2019-02-03,2,0.0,0.00,3
1,2019-02-04,3,0.0,0.00,4
2,2019-02-05,1,0.0,0.00,1
3,2019-02-06,9,2.0,4.01,9
4,2019-02-11,2,0.0,0.00,3
5,2019-02-12,3,0.0,0.00,4
6,2019-02-13,2,0.0,0.00,2
7,2019-02-14,7,0.0,0.00,7
8,2019-02-15,5,0.0,0.00,5
9,2019-02-16,8,2.0,6.80,8


In [ ]:
flood_days=third_tw[third_tw['Floods Frequence']!=0]
no_flood=third_tw[third_tw['Floods Frequence']==0]
x,y=stats.mannwhitneyu(flood_days['METEO'].array, no_flood['METEO'].array, alternative='greater')
print("The statistics: ", x)
print("The p-value", y)
if y<=0.05:
  print("Reject the null hypothesis, that the probability of X being greater than Y is equal to the probability of Y being greater than X")
else:
  print("The null hypothesis cannot be rejected")

The statistics:  199.0
The p-value 0.021690115767079427
Reject the null hypothesis, that the probability of X being greater than Y is equal to the probability of Y being greater than X


In [ ]:
third_tw[third_tw['Floods Frequence']!=0]

,index,Tweets Frequence,Floods Frequence,Duration,METEO
3,2019-02-06,9,2.0,4.01,9
9,2019-02-16,8,2.0,6.80,8
14,2019-02-21,2,7.0,12.72,2
17,2019-02-24,4,1.0,1.75,4
23,2019-03-02,6,3.0,1.59,6
26,2019-03-05,2,5.0,13.56,2
28,2019-03-07,8,6.0,9.39,9
29,2019-03-09,11,2.0,0.60,12


#Last Window

In [ ]:
four_tw=pd.read_csv(r'/content/gdrive/My Drive/Colab Notebooks/Tweets/Fourth_Twindow.csv')
four_tw

,index,Tweets Frequence,Floods Frequence,Duration,METEO
0,2019-01-01,2,1.0,0.82,3
1,2019-01-03,1,3.0,1.03,2
2,2019-01-04,2,1.0,1.63,2
3,2019-01-05,2,0.0,0.00,2
4,2019-01-07,5,0.0,0.00,5
...,...,...,...,...,...
62,2019-03-17,12,0.0,0.00,11
63,2019-03-21,6,0.0,0.00,7
64,2019-03-22,1,0.0,0.00,1
65,2019-03-28,2,0.0,0.00,2


In [ ]:
flood_days=four_tw[four_tw['Floods Frequence']!=0]
no_flood=four_tw[four_tw['Floods Frequence']==0]
x,y=stats.mannwhitneyu(flood_days['METEO'].array, no_flood['METEO'].array, alternative='greater')
print("The statistics: ", x)
print("The p-value", y)
if y<=0.05:
  print("Reject the null hypothesis, that the probability of X being greater than Y is equal to the probability of Y being greater than X")
else:
  print("The null hypothesis cannot be rejected")

The statistics:  558.0
The p-value 0.012948659711990402
Reject the null hypothesis, that the probability of X being greater than Y is equal to the probability of Y being greater than X
